<a href="https://colab.research.google.com/github/arleserp/MinTIC2022/blob/master/25_Django_PyMongo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MinTIC2020 Ciclo 1
Bienvenido al repositorio que contiene el ejercicio de cierre del Ciclo 1 de MisiónTIC 2022 del Gobierno de Colombia.
En este ejercicio usted realizará un paso a paso en donde usando los conocimientos aprendidos en python, nuevas cosas que va a aprender, y algunos elementos previamente construidos por sus profesores, creara un versión bastante sencilla de una Tienda Virtual.
De esta manera, usted podrá ver de una forma directa como utilizar los conceptos que aprendió en este ciclo, además de evidenciar la importancia del trabajo en equipo con otros roles tradicionales (como el caso de los desarrolladores front-end) en la conformación de equipos de desarrollo de software.


## Pre-requisitos 
Tener MongoBD instalado, y la ejecución del Daemon de Mongo durante el tiempo del ejercicio. Adicionalmente, haber ajustado las vistas, plantillas, formularios y mapeo del proyecto de **Django**.



## PyMongo y Transacciones con la Base de Datos
Una vez se tiene la aplicación construida en su base fundamental, lo que incluye el MVC, formularios, y la interacción con el `HTML` solo falta un elemento adicional: las transacciones con la Base de Datos. Es importante tener instalado `MongoDB` para completar el ejercicio, además de tener en ejecuci\'on el `mongod`, o como también se le ha mencionado, el `mongo servidr`.

Por un lado, para la información que se quiera presentar en los `HTML` y se cargue desde la base de datos, las transacciones deben ser realizadas en el archivo `views.py`, en donde están las funciones que conectan las _vistas_ con los _templates_. En este sentido, en la función que corresponda se debe agregar la información obtenida de la base de datos a una lista, que será enviada como un parámetro.

Para este ejercicio, las siguientes son las modificaciones que se deben realizar para completar las transacciones con la base de datos, y por consiguiente, completar la aplicación en su primera versión:
1. Ir a `MongoDB`, mejor conocido como `mongo cliente`, y crear una base de datos para el ejercicio; en este caso, se crea una base de datos de nombre `tiendaVirtual`. En esta base de datos, es necesario que usted cree tres colecciones: (i) _productos_, (ii) _compras_, y (iii) _carrito_. En la consola de `MongoDB` sería algo similar a la siguiente secuencia de instrucciones:
```
use tiendaVirtual
db.createCollection('productos')
db.createCollection('compras')
db.createCollection('carrito')
```
2. Para poblar la base de datos inicialmente, en la carpeta de la aplicación, `tienda_virtual` en este caso, cree un archivo `basedatos_inicial.py`. En este archivo puede generar datos ficticios y almacenarlos en la base de datos, para que su **Tienda Virtual** funcione como usted desea. 
A continuación se le coloca el código que le sirve para conectarse con el cliente de mongo a la base de datos:
```
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db_tienda_virtual = client.tiendaVirtual
```

Este código debe estar al inicio del archivo. Ahora, para crear un listado inicial de productos, en este archivo, podría utilizar un fragmento de código similar al que se presenta a continuación:
```
lista_inicial = [
	{'nombre': "Balón", 'costo': 10000},
	{'nombre': "Chaqueta", 'costo': 85000},
	{'nombre': "Libro", 'costo': 12000},
	{'nombre': "Pantalón", 'costo': 90000},
	{'nombre': "Computador", 'costo': 2500000},
	{'nombre': "Silla", 'costo': 400000},
	{'nombre': "Celular", 'costo': 1250000},
	{'nombre': "Raqueta", 'costo': 250000},
	{'nombre': "Tablet", 'costo': 600000},
	{'nombre': "Mesa", 'costo': 172000},
	{'nombre': "Camiseta", 'costo': 28000},
	{'nombre': "Cámara", 'costo': 2450000},
	{'nombre': "Billetera", 'costo': 62000},
	{'nombre': "Sofá", 'costo': 450000},
	{'nombre': "Alcancia", 'costo': 15000}
]

ids_ = db_tienda_virtual.productos.insert_many(lista_inicial)
print(ids_)
```

3. Invocar al cliente de `PyMongo` y hacer la conexión a la base de datos que usted haya definido en `MongoDB`. Esto debe hacerse antes de la declaración de las funciones dentro del archivo `views.py`, para que esta conexión esté definida en un alcance global del archivo:
```
client = MongoClient('mongodb://localhost:27017/')
db_tienda_virtual = client.tiendaVirtual
```
4. La función `carrito` requiere cargar los elementos que estén en la colección del `carrito`, así que se debe generar el cursos y agregar los elementos a la lista `productos` que ya había sido definida como vacía. Así, usted deberá tener una función similar a la siguiente:
```
def carrito(request):
    '''
    Esta función indica la carga de la pantalla en la cual se visualiza el actual carrito de compras en la Tienda Virtual
    '''
    productos = []
    cursor = db_tienda_virtual['carrito'].find()
    for document in cursor:
        temp = {}
        temp['nombre'] = document['nombre']
        temp['costo'] = document['costo_unidad']
        temp['cantidad'] = document['cantidad']
        productos.append(temp)

    parametros = {'productos':  productos}
    return render(request, 'tienda_virtual/carrito_compras.html', parametros)
``` 
5. Similar al anterior numeral, en la función de `historial` se debe cargar la información contenida en la colección de `compras`. El proceso debe ser similar al de la función `carrito`. A continuación se muestra un ehjermplo de esta función:
```
def historial(request):
    '''
    Esta función indica la carga de la pantalla en la cual se podrá visualizar el historial de compras de la Tienda Virtual
    '''
    historial = []
    cursor = db_tienda_virtual['compras'].find()
    contador = 0
    for document in cursor:
        temp = {}
        temp['numero'] = contador
        temp['nombre'] = document['nombre_cliente']
        temp['costo'] = document['costo']
        temp['productos'] = document['productos']
        temp['fecha'] = document['fecha']
        temp['metodo'] = document['metodo_pago']
        temp['observaciones'] = document['observaciones']
        temp['direccion'] = document['direccion']
        historial.append(temp)
        contador += 1

    parametros = {'historial':  historial}
    return render(request, 'tienda_virtual/historial.html', parametros)
```
6. El formulario llamado _agregar\_producto_ requiere cargar los productos disponibles en la base de datos. Es hora de usar `pymongo` para ello. En ese caso, usted debe instanciar el cliente de `MongoDB`, y obtener la información de la colección `productos`, para luego llenar la lista usando un ciclo `for`. Así, el siguiente sería un ejemplo de cómo quedaría definido el formulario:
```
class agregar_producto(forms.Form):
    '''
    Este formulario corresponde a la opción de agregar productos al carrito de compras.
    '''
    client = MongoClient('mongodb://localhost:27017/')
    db_tienda_virtual = client.tiendaVirtual
    
    cursor = db_tienda_virtual['productos'].find({}, {'nombre': 1})
    productos = ((document['nombre'], document['nombre']) for document in cursor)
    cantidad = ((i, i) for i in range(16))
    
    producto = forms.ChoiceField(label = "Producto", required=True, choices=productos)
    cantidad = forms.ChoiceField(label = "Cantidad", required=True, choices=cantidad)
```

De esta manera, usted ahora ha integrado de manera exitosa `MongoDB` usando la librería `pymongo` con su aplicación de **Django**. Sin embargo, aún debemos hacer un par de pasos adicionales para completar la tarea...pero ya estamos cerca de acabar. 

## Manejo de Formularios
En el archivo `views.py` existen dos vistas que hacen uso de formularios para enviar y recibir información de los templates. En ambos casos se requiere de un tratamiento especial.

Aunque hay variaciones en cada función, el proceso general es similar:
- Verificar si el formulario viene como parte de la petición del HTML, esto quiere decir que alguien diligencio el formulario en el navegador web.
- Si el formulario no viene en la petición, se carga el template normal. En caso contrario, se debe almacenarla información del formulario en una variable de `python`.
- Si el formulario viene en la petición, además de almacenar su valor, se deben extraer sus campos uno por uno, y hacer las operaciones que correspondan contra la base de datos.

De manera particular, se tiene lo siguiente para cada función:
1. La función de `productos` requiere que la información almacenada en la colección de `productos` sea cargada desde la base de datos, y así mostrar esa lista al usuario en la pantalla web. Adicionalmente, se debe verificar si el usuario escogió agregar un producto al carrito, en cuyo caso se valida el formulario, y se guarda el producto junto su costo por unidad y cantidad en el carrito de compras. Un ejemplo de la función `productos` actualiza sería el siguiente:
```
def productos(request):
    '''
    Esta función indica la carga de la pantalla que tiene el listado de productos que pueden ser agregados al carrito de la Tienda Virtual
    '''
    productos = []
    cursor = db_tienda_virtual['productos'].find()
    for document in cursor:
        temp = {}
        temp['nombre'] = document['nombre']
        temp['costo'] = document['costo']
        productos.append(temp)
   
    if request.method == 'POST': 
        frm_agregar = forms.agregar_producto(request.POST) 

        if frm_agregar.is_valid():
            producto = {}
            producto['nombre'] = request.POST.get('producto')
            producto['cantidad'] = request.POST.get('cantidad')
            
            datos_producto = db_tienda_virtual['productos'].find({'nombre': producto['nombre']}, {'costo': 1})
            producto['costo_unidad'] = datos_producto[0]['costo']
            
            db_tienda_virtual['carrito'].insert_one(producto)

    frm_agregar = forms.agregar_producto()
    parametros = {'frm_agregar':  frm_agregar, 'productos': productos}
    return render(request, 'tienda_virtual/lista_productos.html', parametros)
```

2. La función de `pagos` necesita calcular el costo total de los elementos en el carrito para realizar el pago. En este caso, se necesita recorrer cada uno de los elementos en la colección de `carrito`, y calcular el pago total teniendo en cuenta los campos de `costo` y `cantidad`, esto utilizando un ciclo `for`. Adicionalmente, si el usuario ya hizo uso de la opción de pagar, se debe validar el formulario, obtener los campos diligenciados, calcular la fecha actual (se puede usar la librería `datetime` de `python`), y guardar la compra en el historial (además de vaciar el carrito de compras). Un ejemplo de esta función se muestra a continuación:
```
def pagos(request):
    '''
    Esta función indica la carga de la pantalla en la cual se realiza el Pago de los productos que estén en el carrito de la Tienda Virtual
    '''
    costo = 0
    listado_productos = ""

    cursor = db_tienda_virtual['carrito'].find()
    for document in cursor:
        costo += int(document['costo_unidad']) * int(document['cantidad'])
        listado_productos +=  str(document['cantidad']) + ' ' + document['nombre'] + '\n'

    if request.method == 'POST': 
        frm_pago = forms.pagar_carrito(request.POST) 

        if frm_pago.is_valid():
            pago = {}
            pago['productos'] = listado_productos
            pago['metodo_pago'] = request.POST.get('metodo_pago')
            pago['nombre_cliente'] = request.POST.get('nombre')
            pago['direccion'] = request.POST.get('direccion')
            pago['observaciones'] = request.POST.get('observaciones')
            pago['fecha'] = datetime.now().strftime("%d/%m/%Y")
            pago['costo'] = costo
            print(pago)

            costo = 0
            
            db_tienda_virtual['carrito'].drop()
            db_tienda_virtual['compras'].insert_one(pago)

    frm_pago = forms.pagar_carrito()
    
    parametros = {'frm_pago':  frm_pago, 'costo': costo}
    return render(request, 'tienda_virtual/pagar.html', parametros)
```

Para que la captura de la fecha de la compra funcione, debe hacer una importación al inicio del archivo `views.py`, la cual es: `from datetime import datetime`. 

Ejecute en el servidor (`python manage.py runserver`), vaya al navegador y abra su sitio web (`http://127.0.0.1:8000/tienda_virtual/`), y todo está terminado. Felicitaciones, ha logrado tener una primera versión funcional usando `Django` y `MongoDB` de una Tienda Virtual.


# Errores Detectados
### PIP no es reconocido como comando
En este caso, solo requiere instalar de manera específica. Para ello, ejecute las siguientes instrucciones:
```
curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
python get-pip.py
```
Una vez esto se ejecute por completo, pruebe de nuevo el comando _pip_.


### Windows no permite la ejecución de Scripts: 
En este caso, vaya al buscador en Windows (lupa en la parte inferior izquierda), y escriba _Powershell_, allí verá una opción de `Abrir como Administrador`, de click en ella y coloque la siguiente instrucción:
```
Set-ExecutionPolicy RemoteSigned
```
Haga enter, y vuelva y pruebe la activación del entorno virtual, deberá funcionar de manera normal.

### Error en el PATH de Python en Windows
Error típido. Lo puede resolver agregando el path manualmente, o reinstalando `python` como se muestra en https://www.mclibre.org/consultar/python/otros/python-instalacion.html.